<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_02_RNN_LSTM_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

!pip install unidecode
import unidecode
import string
import random
import re
import time, math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
num_epochs = 1000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [ ]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [ ]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

eep'st,
Hourly afflict: merely, thou art death's fool;
For him thou labour'st by thy flight to shun
And yet runn'st toward him still. Thou art not noble;
For all the accommodations that thou bear'st
Ar


In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)#.cuda()

In [ ]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [ ]:
def test(seq):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(seq):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [ ]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        target = label[j]
        y,hidden,cell = model(x,hidden,cell)
        y = y.squeeze()
        target = target.squeeze()
        loss += loss_func(y,target)
    loss.backward()
    optimizer.step()
  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (i) % 50 == 1:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test(200)
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")

------------------------------------------------------
Epoch 1 Loss 918.8874
샘플링을 시작합니다!
------------------------------------------------------
b4woB%}>l~o' '8I&.Q
&Rl-,pl$oR=o;#}	SKI{`e(]L 13u]Oe8bs>5rsX5Ur}DN>;T/?"z!htu-K!KQp#X,%B(V&`D7A	$kb=LgEU#YP7Y8?85
y1TCgU$)&ml\Kb,"_y
s5sGI$X=f}qSun.}V9AXiZeyA<>4HA

------------------------------------------------------
샘플링이 끝났습니다.!
------------------------------------------------------
Epoch 51 Loss 580.4766
샘플링을 시작합니다!
------------------------------------------------------
bpA*[Re mou of ther nie hho wy

I the poony hene o oot:
his aot aneT thwe stertorn ofoot ahot ererie thh thearap Wlgd gal t the rya athyo bed t
m:rd den ith at ghd
 kes ho moorans aeps ioiiomethe hot d

------------------------------------------------------
샘플링이 끝났습니다.!


In [ ]:
print ('------------------------------------------------------')
print("최종 샘플링을 시작합니다!")
print ('------------------------------------------------------')
test(2000)
print('\n')
print ('------------------------------------------------------')
print("최종 샘플링이 끝났습니다.!")

------------------------------------------------------
최종 샘플링을 시작합니다!
------------------------------------------------------
b@or an, at ben ;ond, in th ids the that wond sesr mod ris icon thet ire hond hat idr ther than, sovhecle mimor tofl hiou seist tou rordar low thing thoul at warld wis was y flege, got tou tove siker heneris use sos for freffer of moletd tis tot not orle or wdin waicot oore sucut lpis wind hame, py wornda pare
MTbe my wiou foewe.D

NxI deor pond iseme, ford thou do;
Therifof
Aice your he, Nne we dou fo her fot bhosr.
Ay bcreev,

A:
'ew ton;:
Tou hay fole ind,
URUDCIOE
AD Chind aves rove an, hind ton woe swroms to th in tor bhike lim hone locd He meld me ree, thin th omo

VR
Ans thew, fowin sour gero cooth I yund pas mer faned these, nom thanth hparton ederte en dto frand dur uang novset berend ous fon lout hwedr der
Ghlot withomy and as hesind dor,

Whes anmy worsmee yourte pThean kerut malls?
Whe adee althe,
I ord thaecof thikler lond fare hito mon oou the, fe
: